In [128]:
import pandas as pd 
import numpy as np
import difflib as diff 
import ast

In [129]:
df = pd.read_csv("preprocessed_train.csv",keep_default_na=False)
del df['Unnamed: 0']
df.head()

,textID,text,selected_text,sentiment,text_split,selected_text_split,initial_indice
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"['id', 'have', 'responded', 'if', 'i', 'were',...","['id', 'have', 'responded', 'if', 'i', 'were',...",
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,"['soon', 'sad', 'i', 'will', 'miss', 'you', 'h...","['soon', 'sad']",
2,088c60f138,my boss is bullying me...,bullying me,negative,"['my', 'boss', 'is', 'bullying', 'me']","['bullying', 'me']",
3,9642c003ef,what interview! leave me alone,leave me alone,negative,"['what', 'interview', 'leave', 'me', 'alone']","['leave', 'me', 'alone']",
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"['sons', 'of', '<curse>', 'why', 'couldnt', 't...","['sons', 'of', '<curse>']",


The next step in this process is to setup the start and end indices that the model will be trained on. We can set this up using either character indices or word indices,we need to decide which. It doesnt make a ton of sense to predit indices that cut a word in half so doing this based on characters probably wont work

As we moved onto finding the indi``ces we noticed a new error: Our split text columns are not lists, they are strings made to look like a list. When we saved to csv for some reason it converted the lists to strings. We are going to use a library called ast to fix this issue. We plan to literally evaluate each string as a list. The library will read in the string i.e "[1,2,4]" and convert it to [1,2,3] which is usable viable python code. Note: This is usually a terrible terrible idea and we acknowledge this. It is however the simplest solution to the issue we are having and as this is a low stakes evironment primarily for learning we accept the usage of literal_eval. For more information read the following [link](https://stackoverflow.com/questions/1832940/why-is-using-eval-a-bad-practice)

In [130]:
df["text_split"] = df.text_split.apply(lambda x: ast.literal_eval(x))
df["selected_text_split"] = df.selected_text_split.apply(lambda x: ast.literal_eval(x))
df.head()

,textID,text,selected_text,sentiment,text_split,selected_text_split,initial_indice
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"[id, have, responded, if, i, were, going]","[id, have, responded, if, i, were, going]",
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,"[soon, sad, i, will, miss, you, here, in, san,...","[soon, sad]",
2,088c60f138,my boss is bullying me...,bullying me,negative,"[my, boss, is, bullying, me]","[bullying, me]",
3,9642c003ef,what interview! leave me alone,leave me alone,negative,"[what, interview, leave, me, alone]","[leave, me, alone]",
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"[sons, of, <curse>, why, couldnt, they, put, t...","[sons, of, <curse>]",


Unsurprisingly this data is pretty dirty, so there are plenty of instances where the selected text is really different from the main text. To deal with this we have told get_close_matches to have no cutoff, just return the closest match regardless of similarity.

In [131]:
def index_finder(text,selected,index):
    text = list(text)
    selected = list(selected)
    return text.index(diff.get_close_matches(selected[index],text,cutoff=0)[0])
df["initial_indice"] = df.apply(lambda x: index_finder(x.text_split,x.selected_text_split,0),axis=1)
df["final_indice"] = df.apply(lambda x: index_finder(x.text_split,x.selected_text_split,-1),axis=1)
df.head()

,textID,text,selected_text,sentiment,text_split,selected_text_split,initial_indice,final_indice
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"[id, have, responded, if, i, were, going]","[id, have, responded, if, i, were, going]",0,6
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,"[soon, sad, i, will, miss, you, here, in, san,...","[soon, sad]",0,1
2,088c60f138,my boss is bullying me...,bullying me,negative,"[my, boss, is, bullying, me]","[bullying, me]",3,4
3,9642c003ef,what interview! leave me alone,leave me alone,negative,"[what, interview, leave, me, alone]","[leave, me, alone]",2,4
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"[sons, of, <curse>, why, couldnt, they, put, t...","[sons, of, <curse>]",0,2


In [132]:
df[df.initial_indice > df.final_indice]`

0         True
1         True
2         True
3         True
4         True
         ...  
27476    False
27477     True
27478     True
27479     True
27480     True
Length: 27481, dtype: bool

The issue here is that in places where the selected text is particularly messy the intial index is actually after the selected index. This is a big issue as we absolutely do not want to be telling our model that it is ok to label and initial index as a final index. Since there were only 293 times where that happens which is relatively few we will choose to drop those samples from the dataset 

In [133]:
df = df[df.initial_indice < df.final_indice]
df[df.initial_indice > df.final_indice]

,textID,text,selected_text,sentiment,text_split,selected_text_split,initial_indice,final_indice


In [ ]:
Now there are none that violate that condition!

We can now move to tokenization and preparing the code for modeling!

In [134]:
df.to_csv("feature_engineered.csv")